In [1]:
# Install if you have never used these: unblock the lines below to install if needed

#!pip install webdriver-manager
#!pip3 install lxml
#!pip3 install selenium
#!pip3 install webdriver_manager
#!pip install --upgrade pip
#!pip install -U selenium

   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   ------ --------------------------------- 1.5/10.0 MB 32.7 MB/s eta 0:00:01
   -------------- ------------------------- 3.7/10.0 MB 47.3 MB/s eta 0:00:01
   ----------------------------- ---------- 7.4/10.0 MB 52.1 MB/s eta 0:00:01
   ---------------------------------------  10.0/10.0 MB 58.1 MB/s eta 0:00:01
   ---------------------------------------- 10.0/10.0 MB 49.2 MB/s eta 0:00:00
  Attempting uninstall: selenium
    Found existing installation: selenium 4.17.2
    Uninstalling selenium-4.17.2:
      Successfully uninstalled selenium-4.17.2


In [2]:
# --------- import necessary modules -------

# For webscraping
from bs4 import BeautifulSoup

# Parsing and creating xml data
from lxml import etree as et

# Store data as a csv file written out
from csv import writer

# In general to use with timing our function calls to Indeed
import time

# Assist with creating incremental timing for our scraping to seem more human
from time import sleep

# Dataframe stuff
import pandas as pd

# Random integer for more realistic timing for clicks, buttons and searches during scraping
from random import randint

# Multi Threading
import threading

# Threading:
from concurrent.futures import ThreadPoolExecutor, wait

In [3]:
import selenium

# Check version I am running
selenium.__version__

'4.18.1'

In [4]:
# Selenium 4:

from selenium import webdriver

# Starting/Stopping Driver: can specify ports or location but not remote access
from selenium.webdriver.chrome.service import Service as ChromeService

# Manages Binaries needed for WebDriver without installing anything directly
from webdriver_manager.chrome import ChromeDriverManager

In [6]:
# Allows searchs similar to beautiful soup: find_all
from selenium.webdriver.common.by import By

# Try to establish wait times for the page to load
from selenium.webdriver.support.ui import WebDriverWait

# Wait for specific condition based on defined task: web elements, boolean are examples
from selenium.webdriver.support import expected_conditions as EC

# Used for keyboard movements, up/down, left/right,delete, etc
from selenium.webdriver.common.keys import Keys

# Locate elements on page and throw error if they do not exist
from selenium.common.exceptions import NoSuchElementException

In [118]:
import csv
from time import sleep
from random import randint
from datetime import datetime

job_ = ['business+analyst','data+analyst','software+engineer']
loc = 'Singapore'

paginaton_url_ = 'https://sg.indeed.com/jobs?q={}&l={}&sort=date&start={}'

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
p_ = []
salary_list_ = []
company_list_ = []
location_list_ = []
job_description_list_ = []
posted_date_list_ = []

# Loop through each job and location combination
for job in job_:
    for i in range(0, 7):  # Scrape 7 pages for each job for latest job postings
        driver.get(paginaton_url_.format(job, loc, i * 10))
        sleep(randint(2, 5))

        # Scroll through the page and obtain job listings one by one
        job_page = driver.find_element(By.ID, "mosaic-jobResults")
        jobs = job_page.find_elements(By.CLASS_NAME, "job_seen_beacon")

        for idx, jj in enumerate(jobs):
                    try:
                        # Scroll to the current job listing
                        driver.execute_script("arguments[0].scrollIntoView(true);", jj)
                        sleep(1)

                        # Click on the job listing
                        jj.click()
                        sleep(2)  # Add a short pause after clicking

                    except:
                        close_button = driver.find_element(By.CSS_SELECTOR, "button[aria-label='close']")
                        close_button.click()
                        sleep(1)  # Add a short pause after closing the pop-up

                    # job title
                    try:
                        job_title = driver.find_element(By.CLASS_NAME, "jobsearch-JobInfoHeader-title-container").text
                        p_.append(job_title)
                    except:
                        p_.append(None)
                        
                    # salary
                    try:
                        salary = driver.find_element(By.CSS_SELECTOR, "div[data-testid='jobsearch-OtherJobDetailsContainer']").text
                        salary_list_.append(salary)
                    except:
                        salary_list_.append(None)
            

                    # company name
                    try:
                        company = driver.find_element(By.CSS_SELECTOR, "div[data-testid='inlineHeader-companyName']").text
                        company_list_.append(company)
                    except:
                        company_list_.append(None)    

                    # company location
                    try:
                        location = driver.find_element(By.CSS_SELECTOR, "div[data-testid='inlineHeader-companyLocation']").text
                        location_list_.append(location)
                    except:
                        location_list_.append(None)

                    # job description
                    try:
                        job_description = driver.find_element(By.ID, "jobDescriptionText").text
                        job_description_list_.append(job_description)
                    except:
                        job_description_list_.append(None)
                    
                    # posting date
                    try:
                        posted_date_element = driver.find_element(By.CSS_SELECTOR, "span[data-testid='myJobsStateDate']")
                        posted_date = posted_date_element.get_attribute("textContent")
                        posted_date_list_.append(posted_date)
                    except:
                        posted_date_list_.append(None)
                    
            
driver.quit()

# Write data to CSV file
with open('indeed_jobs_raw.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Job Title', 'Company', 'Salary', 'Job Type', 'Location', 'Job Description', 'Posted Date', 'Today Date', 'Listing Date']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    today_date = datetime.now()
    today_date = today_date.strftime("%d/%m/%Y")
    
    writer.writeheader()
    for job, company, salary, location, job_description, posted_date in zip(p_, company_list_, salary_list_, location_list_, job_description_list_, posted_date_list_):
        writer.writerow({'Job Title': job, 'Company': company, 'Salary': salary, 'Location': location, 'Job Description': job_description, 'Posted Date': posted_date, 'Today Date': today_date})


In [119]:
data = pd.read_csv('indeed_jobs.csv')
data

,Job Title,Company,Salary,Job Type,Location,Job Description,Posted Date,Today Date,Listing Date
0,Senior Business Analyst (Avaloq)\n- job post,PENTAFIN PTE. LTD.,"$11,000 - $12,500 a month - Permanent, Full-time",NaN,Singapore,About Pentafin\nPentafin exists to making bank...,PostedJust posted,21/03/2024,NaN
1,Data Analyst\n- job post,SALT TALENT SEARCH PTE. LTD.,"$5,000 - $5,700 a month - Full-time, Contract",NaN,Singapore,Salt is looking for a Data Analyst to join the...,PostedJust posted,21/03/2024,NaN
2,Business Analyst/QA – Cash Management (Payment...,Crédit Agricole CIB,NaN,NaN,Singapore,Description du poste\nMain responsibilities\n\...,PostedJust posted,21/03/2024,NaN
3,System Analyst- Market Data\n- job post,PERSOLKELLY,"Up to $10,500 a month - Temporary",NaN,Central Singapore,The System Analyst - Market Data will be respo...,PostedJust posted,21/03/2024,NaN
4,"Project Manager/Business Analyst, Fiscal Trans...",ANTAES ASIA PTE. LTD.,"$10,000 - $13,000 a month - Permanent",NaN,Singapore,Job Description:\nContribute to IT projects in...,PostedJust posted,21/03/2024,NaN
...,...,...,...,...,...,...,...,...,...
310,Backend software engineer-Emerging Market\n- j...,NodeFlair,Permanent,NaN,Singapore,Job Summary\n\nJob Type\nPermanent\n\nSeniorit...,PostedPosted 1 day ago,21/03/2024,NaN
311,Site Reliability Engineer\n- job post,TRINITY CONSULTING SERVICES PTE. LTD.,"$3,500 - $6,000 a month - Full-time",NaN,Singapore,Minimum 3 years relevant experience\nStrong te...,PostedPosted 1 day ago,21/03/2024,NaN
312,Site Reliability Engineer (SRE) Only for Fresh...,NTT SINGAPORE PTE. LTD.,"$3,000 - $5,000 a month - Contract",NaN,Singapore,Site Reliability Engineer (SRE)\n\nIntroductio...,PostedPosted 1 day ago,21/03/2024,NaN
313,Lead Software Engineer (Java)\n- job post,Activate Interactive Pte Ltd,"Up to $12,000 a month - Full-time",NaN,Singapore 408600,Activate Interactive Pte Ltd (“Activate”) is a...,PostedPosted 1 day ago,21/03/2024,NaN


## PREPROCESSING FOR INDEED

In [120]:
# Remove additional text of -job post in Job Title Col
for index, row in data.iterrows():
    parts = row['Job Title'].split('-', 1)
    data.loc[index, 'Job Title'] = parts [0]

data

,Job Title,Company,Salary,Job Type,Location,Job Description,Posted Date,Today Date,Listing Date
0,Senior Business Analyst (Avaloq)\n,PENTAFIN PTE. LTD.,"$11,000 - $12,500 a month - Permanent, Full-time",NaN,Singapore,About Pentafin\nPentafin exists to making bank...,PostedJust posted,21/03/2024,NaN
1,Data Analyst\n,SALT TALENT SEARCH PTE. LTD.,"$5,000 - $5,700 a month - Full-time, Contract",NaN,Singapore,Salt is looking for a Data Analyst to join the...,PostedJust posted,21/03/2024,NaN
2,Business Analyst/QA – Cash Management (Payment...,Crédit Agricole CIB,NaN,NaN,Singapore,Description du poste\nMain responsibilities\n\...,PostedJust posted,21/03/2024,NaN
3,System Analyst,PERSOLKELLY,"Up to $10,500 a month - Temporary",NaN,Central Singapore,The System Analyst - Market Data will be respo...,PostedJust posted,21/03/2024,NaN
4,"Project Manager/Business Analyst, Fiscal Trans...",ANTAES ASIA PTE. LTD.,"$10,000 - $13,000 a month - Permanent",NaN,Singapore,Job Description:\nContribute to IT projects in...,PostedJust posted,21/03/2024,NaN
...,...,...,...,...,...,...,...,...,...
310,Backend software engineer,NodeFlair,Permanent,NaN,Singapore,Job Summary\n\nJob Type\nPermanent\n\nSeniorit...,PostedPosted 1 day ago,21/03/2024,NaN
311,Site Reliability Engineer\n,TRINITY CONSULTING SERVICES PTE. LTD.,"$3,500 - $6,000 a month - Full-time",NaN,Singapore,Minimum 3 years relevant experience\nStrong te...,PostedPosted 1 day ago,21/03/2024,NaN
312,Site Reliability Engineer (SRE) Only for Fresh...,NTT SINGAPORE PTE. LTD.,"$3,000 - $5,000 a month - Contract",NaN,Singapore,Site Reliability Engineer (SRE)\n\nIntroductio...,PostedPosted 1 day ago,21/03/2024,NaN
313,Lead Software Engineer (Java)\n,Activate Interactive Pte Ltd,"Up to $12,000 a month - Full-time",NaN,Singapore 408600,Activate Interactive Pte Ltd (“Activate”) is a...,PostedPosted 1 day ago,21/03/2024,NaN


In [121]:
# To seperate the salary and Job Type during web scraping
for index, row in data.iterrows():
    if row['Salary'] == "nan":
        pass
    elif '$' not in str(row['Salary']):
        data.loc[index, 'Job Type'] = row['Salary']
        data.loc[index, 'Salary'] = None
    elif "Up to" in row['Salary'] and row['Salary'].count('-') >= 1:
        parts = row['Salary'].split('-', 1)
        data.loc[index, 'Salary'] = parts[0] 
        data.loc[index, 'Job Type'] = parts[1]
    elif row['Salary'].count('-') >= 2:
        parts = row['Salary'].split('-', 2)
        data.loc[index, 'Salary'] = parts[0] + '-' + parts[1]
        data.loc[index, 'Job Type'] = parts[2]

data

$11,000 - $12,500 a month - Permanent, Full-time
$5,000 - $5,700 a month - Full-time, Contract
nan
Up to $10,500 a month - Temporary
$10,000 - $13,000 a month - Permanent
Permanent
Permanent, Contract
$6,000 - $9,000 a month - Contract
$4,800 - $6,400 a month - Permanent, Full-time
$3,500 - $4,200 a month - Permanent, Full-time
$3,200 - $4,200 a month - Permanent, Full-time
$88,520 - $115,079 a year - Part-time, Full-time
Contract
Up to $8,000 a month - Contract
Up to $8,000 a month - Contract
Full-time
Full-time
Full-time
Permanent, Full-time
Permanent
nan
Full-time, Internship
$6,500 - $9,500 a month - Contract
Internship
$6,000 - $10,000 a month - Permanent, Full-time
Full-time
nan
$4,200 - $6,500 a month - Permanent
nan
$4,000 - $6,000 a month - Full-time
nan
$5,000 - $7,000 a month - Contract
$6,500 - $8,500 a month - Full-time, Contract
$6,500 - $8,500 a month - Full-time, Contract
$8,000 - $8,500 a month - Full-time
Permanent
Permanent
Permanent, Full-time
$5,000 - $9,000 a mont

,Job Title,Company,Salary,Job Type,Location,Job Description,Posted Date,Today Date,Listing Date
0,Senior Business Analyst (Avaloq)\n,PENTAFIN PTE. LTD.,"$11,000 - $12,500 a month","Permanent, Full-time",Singapore,About Pentafin\nPentafin exists to making bank...,PostedJust posted,21/03/2024,NaN
1,Data Analyst\n,SALT TALENT SEARCH PTE. LTD.,"$5,000 - $5,700 a month","Full-time, Contract",Singapore,Salt is looking for a Data Analyst to join the...,PostedJust posted,21/03/2024,NaN
2,Business Analyst/QA – Cash Management (Payment...,Crédit Agricole CIB,None,NaN,Singapore,Description du poste\nMain responsibilities\n\...,PostedJust posted,21/03/2024,NaN
3,System Analyst,PERSOLKELLY,"Up to $10,500 a month",Temporary,Central Singapore,The System Analyst - Market Data will be respo...,PostedJust posted,21/03/2024,NaN
4,"Project Manager/Business Analyst, Fiscal Trans...",ANTAES ASIA PTE. LTD.,"$10,000 - $13,000 a month",Permanent,Singapore,Job Description:\nContribute to IT projects in...,PostedJust posted,21/03/2024,NaN
...,...,...,...,...,...,...,...,...,...
310,Backend software engineer,NodeFlair,None,Permanent,Singapore,Job Summary\n\nJob Type\nPermanent\n\nSeniorit...,PostedPosted 1 day ago,21/03/2024,NaN
311,Site Reliability Engineer\n,TRINITY CONSULTING SERVICES PTE. LTD.,"$3,500 - $6,000 a month",Full-time,Singapore,Minimum 3 years relevant experience\nStrong te...,PostedPosted 1 day ago,21/03/2024,NaN
312,Site Reliability Engineer (SRE) Only for Fresh...,NTT SINGAPORE PTE. LTD.,"$3,000 - $5,000 a month",Contract,Singapore,Site Reliability Engineer (SRE)\n\nIntroductio...,PostedPosted 1 day ago,21/03/2024,NaN
313,Lead Software Engineer (Java)\n,Activate Interactive Pte Ltd,"Up to $12,000 a month",Full-time,Singapore 408600,Activate Interactive Pte Ltd (“Activate”) is a...,PostedPosted 1 day ago,21/03/2024,NaN


In [122]:
# To indicate the listing date from posted date information
import re
from datetime import datetime, timedelta

# Convert text to datetime for manipulation 
data['Today Date'] = pd.to_datetime(data['Today Date'])

for index, row in data.iterrows():
    if not re.search(r'\d', row['Posted Date']):
        data.loc[index, 'Listing Date'] = row['Today Date']

    else:
        days = re.findall(r'\d+', row['Posted Date'])
        days_int = sum(map(int, days))
        listing_date = row['Today Date'] - timedelta(days=days_int)
        data.loc[index, 'Listing Date'] = listing_date

data

,Job Title,Company,Salary,Job Type,Location,Job Description,Posted Date,Today Date,Listing Date
0,Senior Business Analyst (Avaloq)\n,PENTAFIN PTE. LTD.,"$11,000 - $12,500 a month","Permanent, Full-time",Singapore,About Pentafin\nPentafin exists to making bank...,PostedJust posted,2024-03-21,2024-03-21 00:00:00
1,Data Analyst\n,SALT TALENT SEARCH PTE. LTD.,"$5,000 - $5,700 a month","Full-time, Contract",Singapore,Salt is looking for a Data Analyst to join the...,PostedJust posted,2024-03-21,2024-03-21 00:00:00
2,Business Analyst/QA – Cash Management (Payment...,Crédit Agricole CIB,None,NaN,Singapore,Description du poste\nMain responsibilities\n\...,PostedJust posted,2024-03-21,2024-03-21 00:00:00
3,System Analyst,PERSOLKELLY,"Up to $10,500 a month",Temporary,Central Singapore,The System Analyst - Market Data will be respo...,PostedJust posted,2024-03-21,2024-03-21 00:00:00
4,"Project Manager/Business Analyst, Fiscal Trans...",ANTAES ASIA PTE. LTD.,"$10,000 - $13,000 a month",Permanent,Singapore,Job Description:\nContribute to IT projects in...,PostedJust posted,2024-03-21,2024-03-21 00:00:00
...,...,...,...,...,...,...,...,...,...
310,Backend software engineer,NodeFlair,None,Permanent,Singapore,Job Summary\n\nJob Type\nPermanent\n\nSeniorit...,PostedPosted 1 day ago,2024-03-21,2024-03-20 00:00:00
311,Site Reliability Engineer\n,TRINITY CONSULTING SERVICES PTE. LTD.,"$3,500 - $6,000 a month",Full-time,Singapore,Minimum 3 years relevant experience\nStrong te...,PostedPosted 1 day ago,2024-03-21,2024-03-20 00:00:00
312,Site Reliability Engineer (SRE) Only for Fresh...,NTT SINGAPORE PTE. LTD.,"$3,000 - $5,000 a month",Contract,Singapore,Site Reliability Engineer (SRE)\n\nIntroductio...,PostedPosted 1 day ago,2024-03-21,2024-03-20 00:00:00
313,Lead Software Engineer (Java)\n,Activate Interactive Pte Ltd,"Up to $12,000 a month",Full-time,Singapore 408600,Activate Interactive Pte Ltd (“Activate”) is a...,PostedPosted 1 day ago,2024-03-21,2024-03-20 00:00:00


In [123]:
data.to_csv('indeed_jobs_modified.csv', index=False)